## 200803 Project Week4 Lyrics

In [76]:
import requests 
import re
from bs4 import BeautifulSoup
import os
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, \
                            recall_score, confusion_matrix, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.under_sampling import RandomUnderSampler, NearMiss
import numpy as np
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.naive_bayes import MultinomialNB #for count data 
import matplotlib.pyplot as plt

### Step 1: Download the URLs of all songs of your favourite artist.
* Go to the page listing all songs of your favourite artist on lyrics.com.
* Copy the URL
* Use the requests module to download that page
* Save the page to a text file
* Open the page in a text editor
* Examine the HTML code and look for links to songs
* Extract all links using Regular Expressions

In [2]:
url = 'https://www.lyrics.com/artist/Prince/5182'
request = requests.get(url)

In [3]:
request.status_code

200

In [4]:
url2 = 'https://www.lyrics.com/artist/Aretha-Franklin/4305'
request2 = requests.get(url2)
request2.status_code

200

In [5]:
# Save the html in a text file and search in an editor
with open('Prince.txt', 'w') as file: 
    file.write(request.text)

In [6]:
# Save the html in a text file and search in an editor
with open('Aretha.txt', 'w') as file: 
    file.write(request2.text)

In [ ]:
# Check if the file was created
#!ls

In [ ]:
# lyrics links examples (Firefox -> cmd + alt + i -> inspector):
# - <a href="/lyric/36601564/Prince/Dear+Mr+Man">Dear Mr Man</a>
# - <a href="/lyric/36168055/Prince/Make-Up">Make-Up</a>
# - <a href="/lyric/36168053/Prince/You%27re+My+Love">You're My Love</a>

### Step 2: Saving the URL links
* Write a loop that goes through all song URLs that you collected previously
* Construct a complete URL
* Test the URL in a browser manually
* Generate a unique file name (using the song name or a number)
* Download each song
* Save each song to a unique file

In [7]:
# Collect all song https - Prince
list_of_urls = []

pattern = "/lyric/\d+/Prince/[^\"]+"
# creates a string:
text = request.text    
prince_songs = re.findall(pattern, text)

for i in range(len(prince_songs)):
    list_of_urls.append('http://www.lyrics.com' + prince_songs[i])

In [8]:
list_of_urls[:10]

['http://www.lyrics.com/lyric/36628200/Prince/Manic+Monday',
 'http://www.lyrics.com/lyric/36628172/Prince/Wouldn%27t+You+Love+to+Love+Me%3F',
 'http://www.lyrics.com/lyric/36652338/Prince/Don%27t+Let+Him+Fool+Ya',
 'http://www.lyrics.com/lyric/36599249/Prince/I+Feel+for+You+%5BAcoustic+Demo%5D',
 'http://www.lyrics.com/lyric/36464781/Prince/Take+Me+With+U',
 'http://www.lyrics.com/lyric/36464770/Prince/Superfunkycalifragisexy',
 'http://www.lyrics.com/lyric/36465778/Prince/When+2+R+in+Love',
 'http://www.lyrics.com/lyric/36601564/Prince/Dear+Mr+Man',
 'http://www.lyrics.com/lyric/36168059/Prince/Sex+Shooter',
 'http://www.lyrics.com/lyric/36168058/Prince/Jungle+Love']

In [9]:
len(list_of_urls)

1539

In [10]:
# Collect all song https - Aretha Franklin
list_of_urls2 = []

pattern2 = "/lyric/\d+/Aretha\+Franklin/[^\"]+"
text2 = request2.text
aretha_songs = re.findall(pattern2, text2)

for i in range(len(aretha_songs)):
    list_of_urls2.append('http://www.lyrics.com' + aretha_songs[i])

In [11]:
list_of_urls2[:10]

['http://www.lyrics.com/lyric/36071424/Aretha+Franklin/I+Say+a+Little+Prayer',
 'http://www.lyrics.com/lyric/36071434/Aretha+Franklin/Think',
 'http://www.lyrics.com/lyric/36050536/Aretha+Franklin/Don%27t+Play+That+Song',
 'http://www.lyrics.com/lyric/36054477/Aretha+Franklin/Respect',
 'http://www.lyrics.com/lyric/36054463/Aretha+Franklin/Think',
 'http://www.lyrics.com/lyric/36111242/Aretha+Franklin/Respect',
 'http://www.lyrics.com/lyric/36111164/Aretha+Franklin/I+Say+a+Little+Prayer',
 'http://www.lyrics.com/lyric/36137173/Aretha+Franklin/I+Say+a+Little+Prayer',
 'http://www.lyrics.com/lyric/36131030/Aretha+Franklin/I+Say+a+Little+Prayer',
 'http://www.lyrics.com/lyric/36037609/Aretha+Franklin/I+Say+a+Little+Prayer']

In [12]:
len(list_of_urls2)

5762

In [ ]:
# Create folders for artists
artists = ['prince', 'aretha_franklin']

# Create target Directory if don't exist
for i in range(len(artists)):
    os.makedirs(os.path.join('/Users/Carina/Documents/SpicedAcademy/Week4/', artists[i]))

In [ ]:
# Save all lyrics as individual files - Prince

for i in range(len(list_of_urls)):
    
    temp_url = list_of_urls[i]
    temp_req = requests.get(temp_url)
    
    pattern3 = "\/lyric\/.+\/.+\/(.+)$"
    with open("prince/Prince--" + re.findall(pattern3, list_of_urls[i])[0] + '.txt',  'w') as file:
        # convert it to a beautifulsoup object
        soup_prince_lyrics = BeautifulSoup(temp_req.text)  # .text creates a string:
        lyrics = soup_prince_lyrics.pre.get_text()   #pre is the relevant tag for lyrics
        file.writelines(lyrics)
file.close()

In [ ]:
# Save all lyrics as individual files with BeautifulSoup - Aretha Franklin

pattern3 = "\/lyric\/.+\/.+\/(.+)$"

for i in range(len(list_of_urls2)):
    
    with open('aretha_franklin/ArethaFranklin--' + re.findall(pattern3, list_of_urls2[i])[0] + '.txt', 'w') as file:
        
        temp_url2 = requests.get(str(list_of_urls2[i]))
        soup_aretha_lyrics = BeautifulSoup(temp_url2.text)
        lyrics2 = soup_aretha_lyrics.pre.get_text()      #pre is the relevant tag for lyrics
        file.writelines(lyrics2)
file.close()

In [13]:
# List of song titles - Prince:

pattern4 = "/lyric/.+/.+/([^\"]+)$"   # works as well: "/lyric/.+/.+/(.+)$"

song_names_prince = []
for i in range(len(list_of_urls)):
    song_names_prince.append(re.findall(pattern4, list_of_urls[i]))

song_names_prince_flat = []
for sublist in song_names_prince:
    for item in sublist:
        song_names_prince_flat.append(item)
song_names_prince_flat[:10]

['Manic+Monday',
 'Wouldn%27t+You+Love+to+Love+Me%3F',
 'Don%27t+Let+Him+Fool+Ya',
 'I+Feel+for+You+%5BAcoustic+Demo%5D',
 'Take+Me+With+U',
 'Superfunkycalifragisexy',
 'When+2+R+in+Love',
 'Dear+Mr+Man',
 'Sex+Shooter',
 'Jungle+Love']

In [14]:
len(song_names_prince_flat)

1539

In [15]:
# List of song titles - Aretha Franklin:

pattern4 = "/lyric/.+/.+/([^\"]+)$"
song_names_aretha = []
for i in range(len(list_of_urls2)):
    song_names_aretha.append(re.findall(pattern4, list_of_urls2[i]))
song_names_aretha[:10]

song_names_aretha_flat = []
for sublist in song_names_aretha:
    for item in sublist:
        song_names_aretha_flat.append(item)
song_names_aretha_flat[:10]

['I+Say+a+Little+Prayer',
 'Think',
 'Don%27t+Play+That+Song',
 'Respect',
 'Think',
 'Respect',
 'I+Say+a+Little+Prayer',
 'I+Say+a+Little+Prayer',
 'I+Say+a+Little+Prayer',
 'I+Say+a+Little+Prayer']

In [16]:
len(song_names_aretha_flat)

5762

### Step 3: Create corpus

In [17]:
path = '/Users/Carina/Documents/SpicedAcademy/Week4/prince/'

In [18]:
path2 = '/Users/Carina/Documents/SpicedAcademy/Week4/aretha_franklin/'

In [19]:
# Corpus: List of strings (['', '', '']) will all words

corpus = []
path_temp = []
corpus2 = []
path_temp2 = []

In [20]:
for i in range(len(song_names_prince)):
    
    path_temp = path + f'Prince--' + str(song_names_prince_flat[i]) + f'.txt'
    doc = open(path_temp).read().lower().split()      #list of words (as strings) for each song
    doc=' '.join(doc)                         # turns list of strings into one string -> one string will all words
    corpus.append(doc)

In [21]:
label_prince = ['Prince'] * len(corpus)
len(label_prince)

1539

In [22]:
for i in range(len(song_names_aretha)):
    
    path_temp2 = path2 + f'ArethaFranklin--' + str(song_names_aretha_flat[i]) + f'.txt'
    doc2 = open(path_temp2).read().lower().split()      #list of words (as strings) for each song
    doc2=' '.join(doc2)                         # turns list of strings into one string -> one string will all words
    corpus2.append(doc2)

In [23]:
label_aretha = ['Aretha Franklin'] * len(corpus2)
len(label_aretha)

5762

In [24]:
df_prince = pd.DataFrame(list(zip(label_prince, corpus)), columns =['Artist', 'Lyrics'])
df_prince.head(5)

,Artist,Lyrics
0,Prince,six o'clock already i was just in the middle o...
1,Prince,look into my big brown eyes tell me what's on ...
2,Prince,1 2 3 don't let him fool ya i know he got a bi...
3,Prince,"baby, baby, when you look at me i get a warm f..."
4,Prince,i can't disguise the pounding of my heart it b...


In [25]:
df_aretha = pd.DataFrame(list(zip(label_aretha, corpus2)), columns =['Artist', 'Lyrics'])
df_aretha.head(5)

,Artist,Lyrics
0,Aretha Franklin,the moment i wake up before i put on my makeup...
1,Aretha Franklin,you better think (think) think about what you'...
2,Aretha Franklin,don't play that song for me 'cause it brings b...
3,Aretha Franklin,"what you want baby, i got it what you need do ..."
4,Aretha Franklin,you better think (think) think about what you'...


In [29]:
df = pd.concat([df_prince, df_aretha], axis=0, ignore_index=True)

In [ ]:
#corpus_complete = corpus.append(corpus2)
#label_complete = label_prince.append(label_aretha)

In [34]:
# remove duplicates in column 'Lyrics':

df = df.drop_duplicates(subset='Lyrics')
#resets index 1-n and drops the previos indicies
df = df.reset_index(drop=True)    

In [32]:
df.groupby('Artist').size()

Artist
Aretha Franklin    435
Prince             554
dtype: int64

In [35]:
df

,Artist,Lyrics
0,Prince,six o'clock already i was just in the middle o...
1,Prince,look into my big brown eyes tell me what's on ...
2,Prince,1 2 3 don't let him fool ya i know he got a bi...
3,Prince,"baby, baby, when you look at me i get a warm f..."
4,Prince,i can't disguise the pounding of my heart it b...
...,...,...
984,Aretha Franklin,"oh it's the sound of a worried, a worried man ..."
985,Aretha Franklin,"(ooh) what you want (ooh) baby, i got (ooh) wh..."
986,Aretha Franklin,tell me if you love me right now tell me if yo...
987,Aretha Franklin,give yourself to jesus) give yourself to jesus...


### Step 4: Train-test split

In [36]:
X = df[['Lyrics']]

In [37]:
y = df['Artist']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((741, 1), (248, 1), (741,), (248,))

### Step 5: Feature Engineering on X_train and X_test

#### Step 5.1 Bang of Words for Tokenization
* transform your corpus into 
    * Count Vectors and 
    * Tf-idf Vectors (Uniqueness-scaled Bag of Words)

#### Step 5.2 Lammetizing to combine word of the same root

#### Step 5.3 Language Model Sequence Generation

### Step 5.1 Bang of Words for Tokenization

In [39]:
X_train['Lyrics'] = X_train['Lyrics'].str.replace('\d+', '')
X_train['Lyrics'] = X_train['Lyrics'].str.replace(',', '')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
# corpus needs to be in list form or as a DataFrame

#corpus = X_train  -> does not work, needs to be real list
#type(corpus)
corpus_list = X_train['Lyrics'].tolist()
type(corpus_list)

list

In [41]:
# Instantiate the count vector

cv = CountVectorizer(stop_words='english')
vec_train = cv.fit_transform(corpus_list)
#cv.get_feature_names()

In [42]:
# count vector results
look1 = pd.DataFrame(vec_train.todense(), columns=cv.get_feature_names())  #index needed?
look1

,aah,aback,abc,abdomen,abide,able,aboard,abort,abound,abraham,...,zannalee,zebedee,zero,zip,zoomin,ça,étiez,étonner,êtes,ṗolice
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Instantiate the Tf-idf vector

tf = TfidfTransformer()
vec2_train = tf.fit_transform(vec_train)

In [44]:
# tfidf vector results
look2 = pd.DataFrame(vec2_train.todense(), columns=cv.get_feature_names())  #index needed?
look2

,aah,aback,abc,abdomen,abide,able,aboard,abort,abound,abraham,...,zannalee,zebedee,zero,zip,zoomin,ça,étiez,étonner,êtes,ṗolice
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.081506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
737,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
738,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
739,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
#This does both steps (count vectorizer and tfidfTransfomer) in one:
vec3 = TfidfVectorizer(stop_words={'english', 'french'})
vec3_train = vec3.fit_transform(corpus_list)
#vec3.get_feature_names()

In [63]:
look3 = pd.DataFrame(vec3_train.todense(), columns=vec3.get_feature_names())
look3

,aah,aback,abc,abdomen,abide,able,aboard,abort,abound,about,...,zannalee,zebedee,zero,zip,zoomin,ça,étiez,étonner,êtes,ṗolice
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030611,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.078757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057699,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031792,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
737,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
738,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
739,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Step 6: Class imbalance
* Undersample the majority class (=Aretha Franklin) or
* Oversample the minority class (=Prince)
* SMOTE: A popular technique for over-sampling that generates new samples by interpolation
* Near Miss: An undersampling method that select examples based on the distance of majority class examples to minority class examples.

In [67]:
df.groupby('Artist').count()

# Aretha: 435
# Prince: 554  -> im balanced dataset

,Lyrics
Artist,
Aretha Franklin,435
Prince,554


In [65]:
len(y_test)

248

In [69]:
vec3_train.shape, y_train.shape, X_train.shape

((741, 6931), (741,), (741, 1))

In [85]:
# Instantiate the RandomUndersampler Model
rus = RandomUnderSampler()

In [86]:
# Resample the data
X_rus, y_rus = rus.fit_resample(vec3_train, y_train)

In [93]:
X_rus.shape, y_rus.shape

((652, 6931), (652,))

In [87]:
# Instantiate the Undersampler 'Near Miss':
nm = NearMiss()


In [88]:
# Resample the data
X_nm, y_nm = nm.fit_resample(vec3_train, y_train)

In [89]:
X_nm.shape, y_nm.shape

((652, 6931), (652,))

In [90]:
# Instantiate the RandomOverSampler Model
ros = RandomOverSampler()

In [91]:
# Resample the data
X_ros, y_ros = ros.fit_resample(vec3_train, y_train)

In [92]:
X_ros.shape, y_ros.shape

((830, 6931), (830,))

In [94]:
# Instantiate the Oversampler SMOTE
sm = SMOTE()
X_sm, y_sm = sm.fit_resample(vec3_train, y_train)

In [95]:
X_sm.shape, y_sm.shape

((830, 6931), (830,))

### Step 7: Train Model

* RandomForest Classifier
* Naive Bayes

In [ ]:
# Random Forest Model on vectorized data and sampled data

In [83]:
rf = RandomForestClassifier()

In [96]:
model_forest = rf.fit(vec3_train, y_train)
model_forest_rus = rf.fit(X_rus, y_rus)
model_forest_nm = rf.fit(X_nm, y_nm)
model_forest_ros = rf.fit(X_ros, y_ros)
model_forest_sm = rf.fit(X_sm, y_sm)

In [ ]:
y_pred_forest_train_vec = model_forest.predict(vec3_train)
y_pred_forest_train_rus = model_forest_rus.predict(X_rus)
y_pred_forest_train_nm = model_forest_nm.predict(X_nm)
y_pred_forest_train_ros = model_forest_ros.predict(X_ros)
y_pred_forest_train_sm = model_forest_sm.predict(X_sm)

In [ ]:
# Naive Bayes Model on vectorized data and sampled data

In [97]:
nb = MultinomialNB(alpha = 1)

In [98]:
model_bayes = nb.fit(vec3_train, y_train)
model_bayes_rus = nb.fit(X_rus, y_rus)
model_bayes_nm = nb.fit(X_nm, y_nm)
model_bayes_ros = nb.fit(X_ros, y_ros)
model_bayes_sm = nb.fit(X_sm, y_sm)

In [ ]:
y_pred_bayes_train_vec = model_bayes.predict(vec3_train)
y_pred_bayes_train_rus = model_bayes_rus.predict(X_rus)
y_pred_bayes_train_nm = model_bayes_nm.predict(X_nm)
y_pred_bayes_train_ros = model_bayes_ros.predict(X_ros)
y_pred_bayes_train_sm = model_bayes_sm.predict(X_sm)

In [ ]:
# Model pipeline:
#cv = CountVectorizer(stop_words='english')
#tf = TfidfTransformer()
#rf = RandomForestClassifier()
#model = make_pipeline(cv, tf, rf)
#model.fit(X_train, y_train)

### Step 6: Test Model on Test Data

In [99]:
# Feature engineering of X_test: 

X_test['Lyrics'] = X_test['Lyrics'].str.replace('\d+', '')
X_test['Lyrics'] = X_test['Lyrics'].str.replace(',', '')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [100]:
# Feature engineering of X_test: 

corpus_list_test = X_test['Lyrics'].tolist()
type(corpus_list_test)

list

In [101]:
# Feature engineering of X_test:     # Do NOT use fit.transform here (as it is a combination, it would fit the X_test!)

vec3_test = vec3.transform(corpus_list_test)

In [102]:
# Using the RandomForest models on Test data set:

y_pred_forest_vec = model_forest.predict(vec3_test)
y_pred_forest_rus = model_forest_rus.predict(vec3_test)
y_pred_forest_nm = model_forest_nm.predict(vec3_test)
y_pred_forest_ros = model_forest_ros.predict(vec3_test)
y_pred_forest_sm = model_forest_sm.predict(vec3_test)

In [103]:
# Using the RandomForest models on Test data set:

y_pred_bayes_vec = model_bayes.predict(vec3_test)
y_pred_bayes_rus = model_bayes_rus.predict(vec3_test)
y_pred_bayes_nm = model_bayes_nm.predict(vec3_test)
y_pred_bayes_ros = model_bayes_ros.predict(vec3_test)
y_pred_bayes_sm = model_bayes_sm.predict(vec3_test)

In [104]:
# One-hot encoding of y_pred because data needs to be nummerical for all metrics but accuracy

y_test_dummy = []
y_pred_forest_vec_dummy = []
y_pred_forest_rus_dummy = []
y_pred_forest_nm_dummy = []
y_pred_forest_ros_dummy = []
y_pred_forest_sm_dummy = []
y_pred_bayes_vec_dummy = []
y_pred_bayes_rus_dummy = []
y_pred_bayes_nm_dummy = []
y_pred_bayes_ros_dummy = []
y_pred_bayes_sm_dummy = []

In [105]:
def artist_dummy(empty_list, y_test):
    ''' converts artist names to 0 and 1
    
        Parameters: artist names
        Returns: 0 and 1 instead of artist names
    '''
    for i in y_test:
        if i == 'Aretha Franklin':
            empty_list.append(0)
        else:
            empty_list.append(1)
    
    #return empty_list


In [106]:
artist_dummy(y_test_dummy, y_test)
artist_dummy(y_pred_forest_vec_dummy, y_pred_forest_vec)
artist_dummy(y_pred_forest_rus_dummy, y_pred_forest_rus)
artist_dummy(y_pred_forest_nm_dummy, y_pred_forest_nm)
artist_dummy(y_pred_forest_ros_dummy, y_pred_forest_ros)
artist_dummy(y_pred_forest_sm_dummy, y_pred_forest_sm)
artist_dummy(y_pred_bayes_vec_dummy, y_pred_bayes_vec)
artist_dummy(y_pred_bayes_rus_dummy, y_pred_bayes_rus)
artist_dummy(y_pred_bayes_nm_dummy, y_pred_bayes_nm)
artist_dummy(y_pred_bayes_ros_dummy, y_pred_bayes_ros)
artist_dummy(y_pred_bayes_sm_dummy, y_pred_bayes_sm)

In [107]:
def print_evaluations(ytrue, ypred, model):
    '''
    Prints evaluation metrics for a specified model.
    '''
    print(f'How does model {model} score:')
    print(f'The accuracy of the model is: {round(accuracy_score(ytrue, ypred), 3)}')
    print(f'The precision of the model is: {round(precision_score(ytrue, ypred), 3)}')
    print(f'The recall of the model is: {round(recall_score(ytrue, ypred), 3)}')
    print(f'The f1-score of the model is: {round(f1_score(ytrue, ypred), 3)}')

###  Comparison between Random Forest and Naive Bayes Model with unbalanced data:

In [108]:
print_evaluations(y_test_dummy, y_pred_forest_vec_dummy, 'RandomForest')

How does model RandomForest score:
The accuracy of the model is: 0.863
The precision of the model is: 0.867
The recall of the model is: 0.892
The f1-score of the model is: 0.879


In [111]:
print_evaluations(y_test_dummy, y_pred_bayes_vec_dummy, 'NaiveBayes')

How does model NaiveBayes score:
The accuracy of the model is: 0.831
The precision of the model is: 0.888
The recall of the model is: 0.799
The f1-score of the model is: 0.841


### RandomForest model results in higher
* accuracy score (0.86 vs 0.83)
* recall score (0.89 vs 0.80)
* F1 score (0.88 vs 0.84)

### Naive Bayes model results in higher
* precision score (0.87 vs 0.89)
* however both model have very high recall score

-> Random Forest!

### Comparison between different balancing techniques and Random Forest model:

In [112]:
print_evaluations(y_test_dummy, y_pred_forest_vec_dummy, 'RandomForest')

How does model RandomForest score:
The accuracy of the model is: 0.863
The precision of the model is: 0.867
The recall of the model is: 0.892
The f1-score of the model is: 0.879


In [113]:
print_evaluations(y_test_dummy, y_pred_forest_rus_dummy, 'RandomForest, undersampled')

How does model RandomForest, undersampled score:
The accuracy of the model is: 0.863
The precision of the model is: 0.867
The recall of the model is: 0.892
The f1-score of the model is: 0.879


In [114]:
print_evaluations(y_test_dummy, y_pred_forest_nm_dummy, 'RandomForest, near miss')

How does model RandomForest, near miss score:
The accuracy of the model is: 0.863
The precision of the model is: 0.867
The recall of the model is: 0.892
The f1-score of the model is: 0.879


In [115]:
print_evaluations(y_test_dummy, y_pred_forest_ros_dummy, 'RandomForest, oversampled')

How does model RandomForest, oversampled score:
The accuracy of the model is: 0.863
The precision of the model is: 0.867
The recall of the model is: 0.892
The f1-score of the model is: 0.879


In [116]:
print_evaluations(y_test_dummy, y_pred_forest_sm_dummy, 'RandomForest, smote')

How does model RandomForest, smote score:
The accuracy of the model is: 0.863
The precision of the model is: 0.867
The recall of the model is: 0.892
The f1-score of the model is: 0.879


### Comparison between different balancing techniques and Naive Bayes model:

In [119]:
print_evaluations(y_test_dummy, y_pred_bayes_vec_dummy, 'NaiveBayes')

How does model NaiveBayes score:
The accuracy of the model is: 0.831
The precision of the model is: 0.888
The recall of the model is: 0.799
The f1-score of the model is: 0.841


In [120]:
print_evaluations(y_test_dummy, y_pred_bayes_rus_dummy, 'NaiveBayes, undersampled')

How does model NaiveBayes, undersampled score:
The accuracy of the model is: 0.831
The precision of the model is: 0.888
The recall of the model is: 0.799
The f1-score of the model is: 0.841


In [121]:
print_evaluations(y_test_dummy, y_pred_bayes_nm_dummy, 'NaiveBayes, near miss')

How does model NaiveBayes, near miss score:
The accuracy of the model is: 0.831
The precision of the model is: 0.888
The recall of the model is: 0.799
The f1-score of the model is: 0.841


In [122]:
print_evaluations(y_test_dummy, y_pred_bayes_ros_dummy, 'NaiveBayes, oversampled')

How does model NaiveBayes, oversampled score:
The accuracy of the model is: 0.831
The precision of the model is: 0.888
The recall of the model is: 0.799
The f1-score of the model is: 0.841


In [123]:
print_evaluations(y_test_dummy, y_pred_bayes_sm_dummy, 'NaiveBayes, smote')

How does model NaiveBayes, smote score:
The accuracy of the model is: 0.831
The precision of the model is: 0.888
The recall of the model is: 0.799
The f1-score of the model is: 0.841


### Step 7: Build command line interface

* clean up code: make functions, make sure it runs top-to-bottom
* Use the Download button in the menu to export the code to a .py file
* Open the file in an editor
    * Clean up unnecessary lines
    * Place all import statements on top
    * Place all constants after the imports (in capitals)
    * Place functions after the constants
    * Check whether the code still runs
* Alternative: terminal: touch lyrics_classifier.py -> copy and paste the python code (before shift M to have everything in one cell)
* add: if __name__ == '__main__' at the bottom of the script but before the output statements
* add: print('Please enter some lyrics: ')
       user_input = input()    
* save changes in Atom
* terminal: python lyrics_classifier -> type in lyrics -> receive prediction  